In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow_addons.layers import CRF

In [ ]:
class MyBiLSTMCRF:
    def __init__(self,vocabSize,maxLen,tagIndexDict,tagSum,sequenceLength=None,vecSize=100):
        self.vocabSize = vocabSize
        self.vecSize = vecSize
        self.maxLen = maxLen
        self.tagSum = tagSum
        self.tagIndexDict = tagIndexDict
        self.sequenceLength = sequenceLength
        self.buildBiLSTMCRF()
    
    def getTransParam(self,y,tagIndexDict):
        self.trainY = y
        yList = self.trainY.tolist()
        transParam = np.zeros([len(list(tagIndexDict.keys())),len(list(tagIndexDict.keys()))])
        for rowI in range(len(yList)):
            for colI in range(len(yList[rowI])-1):
                transParam[yList[rowI][colI]] [yList[rowI][colI+1]]+=1
        for rowI in range(transParam.shape[0]):
            transParam[rowI]=transParam[rowI]/np.sum(transParam[rowI])
        return transParam
    
    def buildBiLSTMCRF(self):
        myModel = Sequential()
        myModel.add(tf.keras.layers.Input(shape=(self.maxLen,)))
        myModel.add(tf.keras.layers.Embedding(self.vocabSize,self.vecSize))
        myModel.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
            self.tagSum,return_sequences=true,activation="tanh"),merge_mode='sum'))
        myModel.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
            self.tagSum,return_sequences=true,activation="softmax"),merge_mode='sum'))
        crf = CRF(self.tagSum,name='crf_layer')
        myModel.add(crf)
        myModel.compile('adam',loss={'crf_layer':crf.get_loss})
        self.myBiLSTMCRF = myModel
        
    def fit(self,x,y,epochs=100,transParam=None):
        if len(y.shape)==3:
            y=np.argmax(y,axis=-1)
        if self.sequenceLength is None:
            self.sequenceLength=[row.shape[0] for row in y]
        self.transParam = self.getTransParam(y,self,tagIndexDict)
        log_dir = "logs"
        tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)
        history = self.myBiLSTMCRF.fit(x,y,epochs=epochs,callbacks=[tensorboard_callback])
        return history
        
    def predict(self,x):
        preArry = self.myBiLSTMCRF.predict(x)
        return preArry

In [ ]:
myModel = MyBiLSTMCRF(vocabsize#词汇量，maxLen#序列最大长度，tagIndexDict#标签序号词典，tagsum#标签总数，sequencelengths#序列长list))
#https://github.com/Timaos123/BiLSTM-CRF                    